# Modélisation d’un réseau BT – Trey
Par M. Buchser & M. Joye

Réalisé lors du semestre n°1 de 2024-2025



### Code of the work done

Import of librairies

In [329]:
import numpy as np
import pandas as pd
import pandapower as pp
import pp_heig_plot as pp_plot
import pp_heig_simulation as pp_sim
from datetime import time

Import the net from the Excel file

In [330]:
net_file_path = "data/trey_power_network.xlsx"
net = pp_sim.load_net_from_xlsx(file_path=net_file_path)
# net

Control that everything is okay

In [331]:
# net.bus
# net.res_line
# net.ext_grid

Plot the net

In [332]:
plot = pp_plot.plot_power_network(net, plot_title="Network of Trey")

Test power flow - plot with constant flow

In [333]:
pp.runpp(net)
pp_plot.plot_powerflow_result(net=net, plot_title="Network of Trey")

In [334]:
# net.res_bus

In [335]:
# net.res_line

In [336]:
# net.res_trafo

2.1.2.1 Identification and modeling of load curves

In [337]:
# Import datas from CSV file
consumer_file_path = "data/liste_des_batiments_vf.csv"
consumer_data = pd.read_csv(consumer_file_path, encoding="latin1")

# Control
# consumer_data

2.1.2.2 Estimation of annual consumption using floor space

In [338]:
# Create a mapping dictionary for consumption (SIA standard)
consumption_sia = {
    "one_housing": {
        "electricity": 17.8 + 0 + 4.2,
        "heating": 23.4,
        "hot_water": 13.5,
    },  # Individual housing
    "multi_housing": {
        "electricity": 21.6 + 0 + 4.2,
        "heating": 14.2,
        "hot_water": 16.9,
    },  # Collective housing
    "farm": {
        "electricity": 6.3 + 0 + 10.5,
        "heating": 11.5,
        "hot_water": 0.9,
    },  # Agricultural building (assimilated to warehouse)
    "church": {
        "electricity": 4.6 + 0 + 20.9,
        "heating": 6.4,
        "hot_water": 7.3,
    },  # Ecclesiastical building (assimilated to performance hall)
    "industrial": {
        "electricity": 16.8 + 50 + 20.5,
        "heating": 9.0,
        "hot_water": 2.4,
    },  # Industrial building (heavy work)
}


# Function to estimate electricity consumption
def estimate_consumption(row):
    building_type = row["building_class"]
    surface = row["Empreinte au sol [m2]"]

    if building_type in consumption_sia:
        conso = consumption_sia[building_type]
        # Calculate electricity consumption (sum of appliances, installations, and lighting)
        electricity_consumption = conso["electricity"] * surface

        # Add thermal energy if heated electrically
        heating_consumption = 0
        if row["heat_source_1"] == "electricity":
            heating_consumption += conso["heating"] * surface
        if row["heatwater_source_1"] == "electricity":
            heating_consumption += conso["hot_water"] * surface

        return electricity_consumption + heating_consumption
    else:
        return 0


# Apply the function to each row
consumer_data["Estimated_Electricity_Consumption_kWh"] = consumer_data.apply(
    estimate_consumption, axis=1
)

In [346]:
# Display the results
consumer_data[
    [
        "Cabinet",
        "building_class",
        "Empreinte au sol [m2]",
        "Estimated_Electricity_Consumption_kWh",
    ]
]

,Cabinet,building_class,Empreinte au sol [m2],Estimated_Electricity_Consumption_kWh
0,CDBT016055,one_housing,569,12518.0
1,CDBT012139,one_housing,81,1782.0
2,CDBT004774,one_housing,147,3234.0
3,CDBT901452,multi_housing,460,11868.0
4,STMT003438,farm,252,4233.6
5,STMT003438,one_housing,334,11857.0
6,N1,farm,1032,17337.6
7,CDBT901604,farm,347,5829.6
8,CDBT012139,one_housing,84,1848.0
9,CDBT004760,multi_housing,303,7817.4


2.1.2.3 Definition of two scenarios of load curve

In [340]:
# Specify the file path
file_path = "data/Load_curves.xlsx"

# List of sheets to read
sheet_names = ["one_housing", "multi_housing", "farm", "church", "industrial"]

# Dictionary to store data from each sheet
load_curves_df = {}

# Read the specified sheets
for sheet in sheet_names:
    load_curves = pd.read_excel(file_path, sheet_name=sheet)

    # Check and clean columns if necessary
    if (
        "time" in load_curves.columns
        and "winter" in load_curves.columns
        and "summer" in load_curves.columns
    ):
        # Store the cleaned data
        load_curves_df[sheet] = load_curves
    else:
        print(f"Sheet {sheet} is missing required columns.")

        # Normalize the 'winter' and 'summer' columns for each building type
for sheet, df in load_curves_df.items():
    # Calculate the correction factor for the 'winter' and 'summer' columns
    winter_corr_fact = df["winter"].sum() * 365 / (4 * 1000 * 1000)
    summer_corr_fact = df["summer"].sum() * 365 / (4 * 1000 * 1000)

    # Normalize the columns
    df["winter"] = df["winter"] / winter_corr_fact
    df["summer"] = df["summer"] / summer_corr_fact

    # Store the normalized DataFrame back into the dictionary
    load_curves_df[sheet] = df

In [341]:
# # Verification of the 1000 kWh/year value
# for sheet, df in load_curves_df.items():
#     # Display the new total for verification
#     new_winter_sum = df["winter"].sum() * 365 / (4 * 1000)
#     new_summer_sum = df["summer"].sum() * 365 / (4 * 1000)
#     print(f"{sheet} - Normalized winter total: {new_winter_sum:.2f} kWh/year")
#     print(f"{sheet} - Normalized summer total: {new_summer_sum:.2f} kWh/year")

In [342]:
# # Display a preview of the data with the sheet name as a title
# for sheet, df in load_curves_df.items():
#     print("=" * 30)  # Separator line
#     print(f"Data from sheet: {sheet}")  # Display the sheet name
#     print("=" * 30)
#     print(df.head())  # Display the first rows of the dataframe
#     print("\n")  # Add spacing for better readability

Import the power profile from the Excel file

In [343]:
# power_profile_file_path = "data/power_profile.xlsx"
# power_profile = pp_sim.load_power_profile_form_xlsx(file_path=power_profile_file_path)

Apply the profile and the net

In [344]:
# apply = pp_sim.apply_power_profile(power_profile)

Simulation

In [345]:
# pp_sim.run_time_simulation(net)